In [11]:
import pandas as pd
import numpy as np
# import yfinance
# from mpl_finance import candlestick_ohlc
import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt

In [12]:
plt.rcParams['figure.figsize'] = [12, 7]

plt.rc('font', size=14) 

In [13]:

name = 'SPY'
# ticker = yfinance.Ticker(name)
# df = ticker.history(interval="1d",start="2020-03-15",end="2020-07-15")
df=pd.read_csv("C:\\Users\Grant\Desktop\work_git\work\data\\1d\ZILUSDT_1d.csv", index_col="Time")

In [14]:
# df['Time'] = pd.to_datetime(df.index)
# df['Time'] = df['Time'].apply(mpl_dates.date2num)

df = df.loc[:,['Open', 'High', 'Low', 'Close']]

In [15]:
df


,Open,High,Low,Close
Time,,,,
2019-05-11,0.01539,0.01688,0.01506,0.01639
2019-05-12,0.01624,0.01682,0.01518,0.01584
2019-05-13,0.01585,0.01712,0.01570,0.01628
2019-05-14,0.01628,0.01796,0.01626,0.01764
2019-05-15,0.01764,0.02039,0.01719,0.02031
...,...,...,...,...
2022-01-29,0.04527,0.04637,0.04474,0.04569
2022-01-30,0.04571,0.04668,0.04370,0.04451
2022-01-31,0.04447,0.04510,0.04232,0.04487


In [ ]:
def isSupport(df,i):
  support = df['Low'][i] < df['Low'][i-1]  and df['Low'][i] < df['Low'][i+1] \
  and df['Low'][i+1] < df['Low'][i+2] and df['Low'][i-1] < df['Low'][i-2]

  return support

def isResistance(df,i):
  resistance = df['High'][i] > df['High'][i-1]  and df['High'][i] > df['High'][i+1] \
  and df['High'][i+1] > df['High'][i+2] and df['High'][i-1] > df['High'][i-2] 

  return resistance

In [ ]:
levels = []
for i in range(2,df.shape[0]-2):
  if isSupport(df,i):
    levels.append((i,df['Low'][i]))
  elif isResistance(df,i):
    levels.append((i,df['High'][i]))

In [ ]:
def plot_all():
  fig, ax = plt.subplots()

  candlestick_ohlc(ax,df.values,width=0.6, \
                   colorup='green', colordown='red', alpha=0.8)

  date_format = mpl_dates.DateFormatter('%d %b %Y')
  ax.xaxis.set_major_formatter(date_format)
  fig.autofmt_xdate()

  fig.tight_layout()

  for level in levels:
    plt.hlines(level[1],xmin=df['Date'][level[0]],\
               xmax=max(df['Date']),colors='blue')
  fig.show()

In [ ]:
plot_all()

In [ ]:
s =  np.mean(df['High'] - df['Low'])

In [ ]:
def isFarFromLevel(l):
  return np.sum([abs(l-x) < s  for x in levels]) == 0

In [ ]:
levels = []
for i in range(2,df.shape[0]-2):
  if isSupport(df,i):
    l = df['Low'][i]

    if isFarFromLevel(l):
      levels.append((i,l))

  elif isResistance(df,i):
    l = df['High'][i]

    if isFarFromLevel(l):
      levels.append((i,l))

In [ ]:
levels

In [ ]:
plot_all()